# Porto Seguro's Safe Drive Prediction

Model that predicts the probability that a driver will initiate an auto insurance claim in the next year.

Nothing ruins the thrill of buying a brand new car more quickly than seeing your new insurance bill. The sting’s even more painful when you know you’re a good driver. It doesn’t seem fair that you have to pay so much if you’ve been cautious on the road for years.

Inaccuracies in car insurance company’s claim predictions raise the cost of insurance for good drivers and reduce the price for bad ones.

### Import Libraries

In [1]:
import pandas as pd      # for data manipulation and analysis
import numpy as np       # for numerical computing
import matplotlib.pyplot as plt     # for data visualization
import matplotlib.dates as mdates   
import seaborn as sns    # for advanced data visualization
import sklearn           # for machine learning algorithms
from sklearn.feature_selection import mutual_info_classif
import statsmodels.api as sm        # for statistical models and tests
import scipy.stats as stats         # for scientific computing and statistics
import random
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from datetime import datetime

# To plot nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
#mpl.rc('axes', labelsize=14)
#mpl.rc('xtick', labelsize=12)
#mpl.rc('ytick', labelsize=12)

from termcolor import colored
import pylab
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Models
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

C:\Users\peter\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



### 1. Data Information

The data comes in the traditional Kaggle form of one training and test file each: ../input/train.csv & ../input/test.csv. 
Each row corresponds to a specific policy holder and the columns describe their features. The target variable is conveniently named target here and it indicates whether this policy holder made an insurance claim in the past.

In the train and test data, features that belong to similar groupings are tagged as such in the feature names (e.g., **ind**, **reg**, **car**, **calc**). In addition, feature names include the postfix **bin** to indicate **binary** features and **cat** to indicate **categorical** features. Features without these designations are either continuous or ordinal. Values of **-1** indicate that the feature was missing from the observation. The target columns signifies whether or not a claim was filed for that policy holder.

### 2. Import Data

In [2]:
# Set the file path and name
train_path = './data/train.csv'
test_path = './data/test.csv'

# Read the CSV file into a Pandas DataFrame
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Copy of datasets
train = train_df
test = test_df

### 3. Auxiliar Functions

In [3]:
# Function to print main info about the dataset

def ds_info(df):
    print(colored('SIZE' , 'blue', attrs=['bold', 'dark']))
    print(df.size)
    print(colored('\nSHAPE' , 'blue', attrs=['bold', 'dark']))
    print(df.shape)
    print(colored('\nINFO' , 'blue', attrs=['bold', 'dark']))
    print(df.info())
    print(colored('\nNULL VALUES' , 'blue', attrs=['bold', 'dark']))
    print(df.isnull().sum())
    print(colored('\nHEAD' , 'blue', attrs=['bold', 'dark']))
    print(df.head())
    print(colored('\nTAIL' , 'blue', attrs=['bold', 'dark']))
    print(df.tail())
    print(colored('\nDESCRIBE - Numerical' , 'blue', attrs=['bold', 'dark']))
    print(df.describe())
    #print(colored('\nDESCRIBE - Categorical' , 'blue', attrs=['bold', 'dark']))
    #print(df.describe(include='object'))
    
    print(colored('\nUNIQUE ENTRIES IN EACH NUMERICAL COLUMN' , 'blue', attrs=['bold', 'dark']))
    # Get number of unique entries in each column with numerical data
    num_cols = [col for col in df.columns if df[col].dtype != "object"]
    num_nunique = list(map(lambda col: df[col].nunique(), num_cols))    
    dn = dict(zip(num_cols, num_nunique))
    # Print number of unique entries by column, in ascending order
    print(sorted(dn.items(), key=lambda x: x[1]))
    
    print(colored('\nUNIQUE ENTRIES IN EACH CATEGORICAL COLUMN' , 'blue', attrs=['bold', 'dark']))
    # Get number of unique entries in each column with categorical data
    object_cols = [col for col in df.columns if df[col].dtype == "object"]
    object_nunique = list(map(lambda col: df[col].nunique(), object_cols))
    dc = dict(zip(object_cols, object_nunique))
    # Print number of unique entries by column, in ascending order
    print(sorted(dc.items(), key=lambda x: x[1]))

    
# Function to parse date 
#def parse_datetime(data):
#    #convert date.datetime to type datetime 
#    data.datetime = pd.to_datetime(data.datetime)
#    # split in columns
#    data['year'] = data['datetime'].dt.year
#    data['month'] = data['datetime'].dt.month
#    data['day'] = data['datetime'].dt.day
#    data['hour'] = data['datetime'].dt.hour
#    data['weekday'] = data['datetime'].dt.weekday
    
    
# Function to detect and print outliers 
#def detect_outliers(data):
#    # mean, standard deviation and 3-sigma of the data
#    mean = np.mean(data)
#    std = np.std(data)
#    threesigma = 3 * std
    
#    # print upper, lower boundary and boxplot
#    sns.boxplot(data, orient="v")
#    plt.show()
#    lower, upper = mean-3*std, mean+3*std
#    print(f"Upper and lower boundary is: {lower}/{upper}")
    
#    # identify outliers and return the outliers
#    outliers = [x for x in data if np.abs(x - mean) > threesigma]
#    print(f"There are {len(outliers)} outliers based on three-sigma rule")

# Function to delete the outliers 
#def delete_outliers(data, df):
#    # detecting and dropping outliers 
#    original_shape = df.shape
#    mean = np.mean(data)
#    std = np.std(data)
#    outliers = np.abs(data-mean) > (3*std)
#    outliers_num = len(train[outliers])
#    df.drop(index=data[outliers].index, inplace=True)
    
#    # print what was deleted
#    print("Number of outliers deleted:", outliers_num)
#    print ("Shape of dataframe with Ouliers: ",original_shape)
#    print ("Shape of Dataframe After Deleting the Ouliers: ",df.shape)

# Function to plot the Correlation Heatmap
def correlation_heatmap(corr):
    mask = np.array(corr)
    mask[np.tril_indices_from(mask)] = False
    
    # plot the correlation
    fig,ax= plt.subplots()
    fig.set_size_inches(10,10)
    sns.heatmap(corr, mask=mask,vmax=.8, square=True,annot=True)

# Function to calculate RMSLE - Root Mean Squared Logarithmic Error   
def rmsle(y_true, y_pred):
    """
    Custom RMSLE scorer function.
    """
    log_diff = np.log1p(y_pred) - np.log1p(y_true)
    rmsle_score = np.sqrt(np.mean(log_diff**2))
    return rmsle_score

# Timer function
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

### 4. Data Overview

In [4]:
ds_info(train)

SIZE
35117508

SHAPE
(595212, 59)

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595212 entries, 0 to 595211
Data columns (total 59 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              595212 non-null  int64  
 1   target          595212 non-null  int64  
 2   ps_ind_01       595212 non-null  int64  
 3   ps_ind_02_cat   595212 non-null  int64  
 4   ps_ind_03       595212 non-null  int64  
 5   ps_ind_04_cat   595212 non-null  int64  
 6   ps_ind_05_cat   595212 non-null  int64  
 7   ps_ind_06_bin   595212 non-null  int64  
 8   ps_ind_07_bin   595212 non-null  int64  
 9   ps_ind_08_bin   595212 non-null  int64  
 10  ps_ind_09_bin   595212 non-null  int64  
 11  ps_ind_10_bin   595212 non-null  int64  
 12  ps_ind_11_bin   595212 non-null  int64  
 13  ps_ind_12_bin   595212 non-null  int64  
 14  ps_ind_13_bin   595212 non-null  int64  
 15  ps_ind_14       595212 non-null  int64  
 16  ps_ind_15       

[('target', 2), ('ps_ind_06_bin', 2), ('ps_ind_07_bin', 2), ('ps_ind_08_bin', 2), ('ps_ind_09_bin', 2), ('ps_ind_10_bin', 2), ('ps_ind_11_bin', 2), ('ps_ind_12_bin', 2), ('ps_ind_13_bin', 2), ('ps_ind_16_bin', 2), ('ps_ind_17_bin', 2), ('ps_ind_18_bin', 2), ('ps_car_08_cat', 2), ('ps_calc_15_bin', 2), ('ps_calc_16_bin', 2), ('ps_calc_17_bin', 2), ('ps_calc_18_bin', 2), ('ps_calc_19_bin', 2), ('ps_calc_20_bin', 2), ('ps_ind_04_cat', 3), ('ps_car_02_cat', 3), ('ps_car_03_cat', 3), ('ps_car_05_cat', 3), ('ps_car_07_cat', 3), ('ps_car_10_cat', 3), ('ps_ind_02_cat', 5), ('ps_ind_14', 5), ('ps_car_11', 5), ('ps_car_09_cat', 6), ('ps_calc_04', 6), ('ps_calc_05', 7), ('ps_ind_01', 8), ('ps_ind_05_cat', 8), ('ps_calc_09', 8), ('ps_reg_01', 10), ('ps_car_04_cat', 10), ('ps_calc_01', 10), ('ps_calc_02', 10), ('ps_calc_03', 10), ('ps_calc_07', 10), ('ps_calc_06', 11), ('ps_calc_08', 11), ('ps_calc_12', 11), ('ps_ind_03', 12), ('ps_car_01_cat', 13), ('ps_ind_15', 14), ('ps_calc_13', 14), ('ps_car_1

In [5]:
ds_info(test)

SIZE
51783328

SHAPE
(892816, 58)

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892816 entries, 0 to 892815
Data columns (total 58 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              892816 non-null  int64  
 1   ps_ind_01       892816 non-null  int64  
 2   ps_ind_02_cat   892816 non-null  int64  
 3   ps_ind_03       892816 non-null  int64  
 4   ps_ind_04_cat   892816 non-null  int64  
 5   ps_ind_05_cat   892816 non-null  int64  
 6   ps_ind_06_bin   892816 non-null  int64  
 7   ps_ind_07_bin   892816 non-null  int64  
 8   ps_ind_08_bin   892816 non-null  int64  
 9   ps_ind_09_bin   892816 non-null  int64  
 10  ps_ind_10_bin   892816 non-null  int64  
 11  ps_ind_11_bin   892816 non-null  int64  
 12  ps_ind_12_bin   892816 non-null  int64  
 13  ps_ind_13_bin   892816 non-null  int64  
 14  ps_ind_14       892816 non-null  int64  
 15  ps_ind_15       892816 non-null  int64  
 16  ps_ind_16_bin   

[('ps_ind_06_bin', 2), ('ps_ind_07_bin', 2), ('ps_ind_08_bin', 2), ('ps_ind_09_bin', 2), ('ps_ind_10_bin', 2), ('ps_ind_11_bin', 2), ('ps_ind_12_bin', 2), ('ps_ind_13_bin', 2), ('ps_ind_16_bin', 2), ('ps_ind_17_bin', 2), ('ps_ind_18_bin', 2), ('ps_car_08_cat', 2), ('ps_calc_15_bin', 2), ('ps_calc_16_bin', 2), ('ps_calc_17_bin', 2), ('ps_calc_18_bin', 2), ('ps_calc_19_bin', 2), ('ps_calc_20_bin', 2), ('ps_ind_04_cat', 3), ('ps_car_02_cat', 3), ('ps_car_03_cat', 3), ('ps_car_05_cat', 3), ('ps_car_07_cat', 3), ('ps_car_10_cat', 3), ('ps_ind_02_cat', 5), ('ps_ind_14', 5), ('ps_car_11', 5), ('ps_car_09_cat', 6), ('ps_calc_04', 6), ('ps_calc_05', 7), ('ps_ind_01', 8), ('ps_ind_05_cat', 8), ('ps_calc_09', 8), ('ps_reg_01', 10), ('ps_car_04_cat', 10), ('ps_calc_01', 10), ('ps_calc_02', 10), ('ps_calc_03', 10), ('ps_calc_06', 10), ('ps_calc_07', 10), ('ps_calc_12', 11), ('ps_ind_03', 12), ('ps_calc_08', 12), ('ps_car_01_cat', 13), ('ps_ind_15', 14), ('ps_car_15', 15), ('ps_calc_13', 16), ('ps_c

Summary:

- The TRAIN dataset contain 595212 rows and 59 columns, while TEST 892816 rows and 58 columns
- The competition host [Adriano Moala](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/40222) explains the names of the variables are based on their origin and they are independent, but unfortunately the meaning cannot be shared:
 - ind: related to the individual
 - reg: related to the region
 - car: related to the car
 - calc: calculated feature
 - bin: indicate binary features
 - cat: indicate categorical features
 - Features without designations are either continuous or ordinal
 - Value of -1 indicate missing values


#### Null or Missing values

In [6]:
train.isnull().any().any()

False

Despite the result showing False for null values, we know from the Porto Seguro explanation that the "values of -1 indicate that the feature was missing from the observation.
Let's look which columns contain -1 values:

In [7]:
train_copy = train
train_copy = train_copy.replace(-1, np.NaN)

In [8]:
# Select columns 1 to 29
columns_to_visualize = train_copy.columns[1:29]

# Create a subset DataFrame with selected columns
subset = train_copy[columns_to_visualize]
# Increase the size of the plot
plt.figure(figsize=(15, 8))
# Generate the heatmap
sns.heatmap(subset.isnull(),yticklabels=False,cbar=False,cmap='cividis')
# Display
plt.show()

KeyboardInterrupt: 

In [ ]:
# Select columns 29 to 59
columns_to_visualize = train_copy.columns[29:59]
# Create a subset DataFrame with selected columns
subset = train_copy[columns_to_visualize]
# Increase the size of the plot
plt.figure(figsize=(15, 8))
# Generate the heatmap
sns.heatmap(subset.isnull(),yticklabels=False,cbar=False,cmap='cividis')
# Display
plt.show()

The heatmaps above demonstrates in yellow that 7 features out of 59 of train dataset contain Null values:
- ps_ind_05_cat
- ps_reg_03
- ps_car_03_cat
- ps_car_05_cat
- ps_car_07_cat
- ps_car_14

The majority of missing values are in features **ps_reg_03**, **ps_car_03_cat and**, **ps_car_05_cat**, where probably more than a half of data is missing.

#### Inspection of Target

As observed previously, Target contain only two values, 0 and 1. Let's visualize them:

In [ ]:
sns.countplot(x='target', data=train)
# Labels and title
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Target')

# Display the plot
plt.show()

In [ ]:
value_counts = train['target'].value_counts()

# Calculate percentages
percentages = (value_counts / len(train)) * 100
print(percentages)

This demonstrate the dataset is imbalanced. Imbalanced datasets pose challenges in machine learning because algorithms can be biased towards the majority class and may struggle to learn patterns or make accurate predictions for the minority class.

### 5. Correlation 

#### Datatype
As shown previously, the dataset is comprised of integers and float types of data. Let's split them to get the correlation between variables:

In [ ]:
train_float = train.select_dtypes(include=['float64'])
train_int = train.select_dtypes(include=['int64'])

#### Float features

In [ ]:
colormap = 'cividis'
plt.figure(figsize=(16,12))
plt.title('Pearson correlation of float features', y=1.03, size=15)
sns.heatmap(train_float.corr(),linewidths=0.1,vmax=1.0, square=True, 
            cmap=colormap, linecolor='white', annot=True)

Analysis:

Most features have correlation near zero or no correlation between them. The only ones returning a positive linear correlation:
- ps_reg_01 and ps_reg_02
- ps_reg_01 and ps_reg_03
- ps_reg_02 and ps_reg_03
- ps_car_12 and ps_car_13
- ps_car_13 and ps_car_15

#### Integer features

Since the number of integer features is much higher than float features, the heatmap will be impossible to read the correlation inside each squere. This is why the heat map chosen is interactive.

In [ ]:
# Date for three different axes
data = [
    go.Heatmap(
        z= train_int.corr().values,
        x=train_int.columns.values,
        y=train_int.columns.values,
        colorscale='cividis',
        reversescale = False,
        opacity = 1.0)
]
# Layout
layout = go.Layout(
    title='Correlation of Integer features',
    xaxis = dict(ticks='', nticks=36),
    yaxis = dict(ticks='' ),
    width = 950, height = 700)

# Plot
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')

Obs: in GitHub this interactive heat map is going to show, but accessing the code in Kaggle is possible to see it.

Analysis:

As the previous correlation, most features have linear correlation near zero or no correlation between them. In this heatmap we can also observe some negatively correlated features:
- ps_ind_07_bin
- ps_ind_06_bin
- ps_ind_16_bin
- ps_ind_17_bin
- ps_ind_18_bin

The hight positive correlation to take note is 
- ps_ind_14 / ps_ind_12_bin: z = 0.89
- ps_ind_14 / ps_ind_11_bin: z = 0.56

An interesting correlation to note is between ps_car_03_cat / ps_car_05_cat (z = 0.489). Both variables contain most of their values missing, and this could explain the reasonable correlation between them. 

This correlation analysis will be relevant if we are going to perform dimensionality reduction transformations such as Principal Component Analysis (PCA), this would require a certain degree of correlation

### Mutual information

Mutual information is a statistical measure that quantifies the dependency or relationship between two variables. In the context of feature selection, mutual_info_classif computes the mutual information between each feature and the target variable, considering the class labels.

The mutual_info_classif method is particularly useful in identifying relevant features for classification problems, especially when the relationship between the features and the target variable is non-linear or complex. It can help in reducing the dimensionality of the feature space, improving model performance, and gaining insights into the data's predictive power.

In [ ]:
# Create the feature matrix
X = train_float  # Feature matrix
y = train['target']

# Compute mutual information scores
mi_scores = mutual_info_classif(X, y)

# Create a DataFrame to display the scores for each float feature
feature_scores = pd.DataFrame({'Feature': train_float.columns, 'Mutual Information': mi_scores})
feature_scores = feature_scores.sort_values('Mutual Information', ascending=False)

# Print the feature scores
print(feature_scores)

In [ ]:
# Create the feature matrix
drop_columns = ['id', 'target']
X = train_int.drop(drop_columns, axis=1).values  # Feature matrix
y = train_int['target']

# Compute mutual information scores
start_time = timer(None)
mi_scores = mutual_info_classif(X, y)
timer(start_time)

In [ ]:
# Get column names
column_names = train_int.drop(drop_columns, axis=1).columns

# Create a DataFrame to display the scores for each integer feature
feature_scores = pd.DataFrame({'Feature': column_names, 'Mutual Information': mi_scores})
feature_scores = feature_scores.sort_values('Mutual Information', ascending=False)

# Print the feature scores
print(feature_scores)

### Feature Importance

Feature importance refers to the process of determining the relevance or importance of features (variables) in a dataset for a given predictive model. Feature importance can help in understanding the impact of each feature on the model's performance and gaining insights into the underlying data patterns

#### Random Forest

In [ ]:
# Load the dataset and split it into features (X) and target variable (y)
X = train.drop(['id', 'target'], axis=1)
y = train['target']

# Split the data into training and validation sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train with Random Forest Classifier
start_time = timer(None)
rfc = RandomForestClassifier(n_estimators=150, max_depth=8, min_samples_leaf=4, max_features=0.2, n_jobs=-1, random_state=42)
rfc.fit(X, y)
timer(start_time)

features = X.columns.values

Let's obtain the list of feature importance using "featureimportances".

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(rfc.feature_importances_, features), 
                                                            reverse = False)))
trace = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'cividis',
        reversescale = True
    ),
    name='Random Forest Feature Importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
    width = 950, height = 1800,
    yaxis=dict(
        showgrid=True,
        showline=False,
        showticklabels=True,
    ))

fig = go.Figure(data=[trace])
fig['layout'].update(layout)
py.iplot(fig, filename='plots')

#### Gradient Boosting Model

In [ ]:
# Load the dataset and split it into features (X) and target variable (y)
X = train.drop(['id', 'target'], axis=1)
y = train['target']

# Split the data into training and validation sets

# Initialize and train with Random Forest Classifier
gbc = GradientBoostingClassifier(n_estimators=100, max_depth=3, min_samples_leaf=4, max_features=0.2, random_state=42)
gbc.fit(X, y)

features = X.columns.values

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(gbc.feature_importances_, features), 
                                                            reverse = False)))
trace = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'cividis',
        reversescale = True
    ),
    name='Gradient Boosting Feature Importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
    width = 950, height = 1800,
    yaxis=dict(
        showgrid=True,
        showline=False,
        showticklabels=True,
    ))

fig = go.Figure(data=[trace])
fig['layout'].update(layout)
py.iplot(fig, filename='plots')

### 6. Feature engineering 

#### Missing Values

The missing values are represented by -1 in the features.

In [ ]:
var_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        var_missing.append(f)
        percentage = missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, percentage))
        
print('There are {} variables with missing values'.format(len(var_missing)))

Analysis:
- Variables **ps_car_03_cat** has 69% and **ps_car_05_cat** has 45% missing values: due to a large proportion of missing values, will be dropped.
- The remaining categorical data will be left with missing value -1
- ps_reg_03 has 18% missing values: use mean to fill out, since is continuous data
- ps_car_14 has 7% missing values: use mean to fill out, since is continuous data
- ps_car_12 has 1 records missing: use mean to fill out, since is continuous data
- ps_car_11 has 5 records with missing values: use mode since it's ordinal data

In [ ]:
# Different approach: replace -1 by NaN)
train = train.replace(-1, np.nan)
test = test.replace(-1, np.nan)

##### Dropping variables

In [ ]:
# Dropping the variables ps_car_03_cat and ps_car_05_cat
#drop_cat_var = ['ps_car_03_cat', 'ps_car_05_cat']
#train.drop(drop_cat_var, inplace=True, axis=1)
#test.drop(drop_cat_var, inplace=True, axis=1)

##### Filling missing values

In [ ]:
# Imputing with the mean
#mean_imputer = SimpleImputer(missing_values=-1, strategy='mean')
#train['ps_reg_03'] = mean_imputer.fit_transform(train[['ps_reg_03']]).ravel()
#train['ps_car_12'] = mean_imputer.fit_transform(train[['ps_car_12']]).ravel()
#train['ps_car_14'] = mean_imputer.fit_transform(train[['ps_car_14']]).ravel()

# Imputing with the mode
#mode_imputer = SimpleImputer(missing_values=-1, strategy='most_frequent')
#train['ps_car_11'] = mode_imputer.fit_transform(train[['ps_car_11']]).ravel()

#### Removing ps_calc variables

As the correlation plot demonstraded, most ps_calc varaibles have zero correlation with any other variable of the data set. Let's test removing them. 

In [ ]:
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)

#### Inspection of Binary, Categorical and Ordinary features

#### Binary feature analysis

Binary features, which have only two possible values (e.g., 0 and 1, True and False), can provide valuable information and play a significant role in a data analysis or machine learning project

In [ ]:
# List of binary columns
bin_col = [col for col in train.columns if '_bin' in col]

# Create lists to store the count of zeros and ones
zero_list = []
one_list = []
for col in bin_col:
    zero_list.append((train[col]==0).sum())
    one_list.append((train[col]==1).sum())

In [ ]:
# Create go.Bar for each list
bar_zero = go.Bar(
    x=bin_col,
    y=zero_list ,
    name='Zero count'
)
bar_one = go.Bar(
    x=bin_col,
    y=one_list,
    name='One count'
)

# Create the layout
data = [bar_zero, bar_one]
layout = go.Layout(
    barmode='stack',
    title='Count binary variables'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='binary-stacked-bar')
plt.savefig('graph/binary-stacked-bar.png')

Analisys:

We can observe that some features are mainly composed by 0, such as:
- ps_ind_10_bin
- ps_ind_11_bin
- ps_ind_12_bin
- ps_ind_13_bin

This observation complements what the previous analysis from mutual information, that these features might not be useful since they are the least relevant features for a classification problem.   

#### Category feature analysis


#### Dummyfication of categorical variables

In [ ]:
# Preprocessing 
id_test = test['id'].values
target_train = train['target'].values

In [ ]:
# Drop columns for final model
drop_columns = ['id', 'target']
X_train = train.drop(drop_columns, axis=1)
X_test = test.drop(['id'], axis=1)

In [ ]:
# Confirm shape
#print(X_train.values.shape, X_test.values.shape)

In [ ]:
cat_features = [a for a in X_test.columns if a.endswith('cat')]

for column in cat_features:
    temp = pd.get_dummies(pd.Series(X_train[column]))
    X_train = pd.concat([X_train,temp],axis=1)
    X_train = X_train.drop([column],axis=1)
    
for column in cat_features:
    temp = pd.get_dummies(pd.Series(X_test[column]))
    X_test = pd.concat([X_test,temp],axis=1)
    X_test = X_test.drop([column],axis=1)

In [ ]:
print(X_train.values.shape, X_test.values.shape)

In [ ]:
#train.describe()

### Model

I would like to acknowledge and give credit to Vladimir Deminov for his excellent code implementation, which served as a valuable reference for my project. 
For those interested in exploring their code, please refer to the following link: https://www.kaggle.com/code/yekenot/simple-stacker-lb-0-284/script.

In [ ]:
# LightGBM params

lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 650
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500
lgb_params['seed'] = 99
lgb_params['n_jobs'] = -1

lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.02
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['seed'] = 99
lgb_params2['n_jobs'] = -1


lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['seed'] = 99
lgb_params3['n_jobs'] = -1

# RandomForest params
#rf_params = {}
#rf_params['n_estimators'] = 200
#rf_params['max_depth'] = 6
#rf_params['min_samples_split'] = 70
#rf_params['min_samples_leaf'] = 30


# ExtraTrees params
#et_params = {}
#et_params['n_estimators'] = 155
#et_params['max_features'] = 0.3
#et_params['max_depth'] = 6
#et_params['min_samples_split'] = 40
#et_params['min_samples_leaf'] = 18


# XGBoost params
#xgb_params = {}
#xgb_params['objective'] = 'binary:logistic'
#xgb_params['learning_rate'] = 0.04
#xgb_params['n_estimators'] = 490
#xgb_params['max_depth'] = 4
#xgb_params['subsample'] = 0.9
#xgb_params['colsample_bytree'] = 0.9  
#xgb_params['min_child_weight'] = 10


# CatBoost params
#cat_params = {}
#cat_params['iterations'] = 900
#cat_params['depth'] = 8
#cat_params['rsm'] = 0.95
#cat_params['learning_rate'] = 0.03
#cat_params['l2_leaf_reg'] = 3.5  
#cat_params['border_count'] = 8
#cat_params['gradient_iterations'] = 4


# Regularized Greedy Forest params
#rgf_params = {}
#rgf_params['max_leaf'] = 2000
#rgf_params['learning_rate'] = 0.5
#rgf_params['algorithm'] = "RGF_Sib"
#rgf_params['test_interval'] = 100
#rgf_params['min_samples_leaf'] = 3 
#rgf_params['reg_depth'] = 1.0
#rgf_params['l2'] = 0.5  
#rgf_params['sl2'] = 0.005

In [ ]:
lgb_model = LGBMClassifier(**lgb_params)

lgb_model2 = LGBMClassifier(**lgb_params2)

lgb_model3 = LGBMClassifier(**lgb_params3)

#rf_model = RandomForestClassifier(**rf_params)

#et_model = ExtraTreesClassifier(**et_params)
        
#xgb_model = XGBClassifier(**xgb_params)

#cat_model = CatBoostClassifier(**cat_params)

#rgf_model = RGFClassifier(**rgf_params) 

#gb_model = GradientBoostingClassifier(max_depth=5)

#ada_model = AdaBoostClassifier()

log_model = LogisticRegression()

In [ ]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [ ]:
start_time = timer(None)
stack = Ensemble(n_splits=3,
        stacker = log_model,
        base_models = (lgb_model, lgb_model2, lgb_model3)) 

y_pred = stack.fit_predict(X_train, target_train, X_test)
timer(start_time)

In [ ]:
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_pred
sub.to_csv('stacked_3.csv', index=False)

BEST SCORE 
- 0.28349 V2 - stacked_1.csv - Fulfilling missing values, standard lgb_model (1 core)
- 0.28349 V3 - stacked_2.csv - Not imputing values to missing values, kept 
- 0.29012 V4 - stacked_2.csv - Same as V3, but removed ps_calc variables 